# Algorithme

In [1]:
import numpy as np
import random 
import time
import csv
from datetime import datetime
from datetime import timedelta
import pandas as pd
from statistics import mean
import logging
import uuid

In [2]:
logging.basicConfig(filename="log_pacing.log", level=logging.INFO)

In [3]:
class Algo:
    def __init__(self, budget_quotidien, nb_heures_jour):
        """Class constructor"""
        # attributs fixes
        self.budget_quotidien = budget_quotidien
        self.cible = budget_quotidien/(nb_heures_jour*3600)
        # On fixe un jour impossible pour initialiser le setup dans les fonctions
        self.jour = 0
    
    def decision_achat_avec_vitesse(self, ts, prix):
        """From a BR, decide whether to buy or not
        
        Arguments:
        :ts: timestamp of the BR
        :prix: price of the BR
        """
        # TS de la br
        day = datetime.fromtimestamp(ts).day
        month = datetime.fromtimestamp(ts).month
        year = datetime.fromtimestamp(ts).year
        # On detecte le changement de jour (reinitialisation du budget restant)
        if self.jour != day:
            logging.info("-\n CHANGING DAY \n")
            self.budget_restant = self.budget_quotidien
            self.BT = [self.cible]
            self.vitesse = pd.DataFrame({'V':0}, index=[datetime(year,month,day,6,0,0,0)])
            self.budget_engage = 0
            self.budget_depense = 0
        self.jour = day
        # On calcule le temps restant avant la fin de la journée
        end_day = datetime(year,month,day,19,40,0,0)
        temps_restant =  datetime.timestamp(end_day) - ts
        # calcul de bt
        created_time = self.vitesse.index[-1] - timedelta(minutes=30)
        self.budget_restant = self.budget_quotidien - (self.budget_engage + self.budget_depense)
        bt = self.budget_restant * ((1+100*self.vitesse.V[self.vitesse.index > created_time].mean()) / temps_restant) 
        self.BT.append(bt)
        # Calcul de vt
        vt = self.BT[-1] - self.BT[-2] 
        self.vitesse = self.vitesse.append(pd.DataFrame({'V':vt}, index=[datetime.fromtimestamp(ts)]))
        if (bt >= self.cible or bt<=0) and (self.budget_restant - prix) >= 0:
            achat = True
            self.budget_engage += prix
        else:
            achat = False

        return (achat, bt, vt)     

    def send_pending_notifications(self,current_ts = None):
        """ Send notifications 
        
        :param current_ts: if None: will send all notifications, else send before current_ts
        :return:
        """
        while len(pending_notifications) > 0 and (pending_notifications[0]['timestamp'] <= current_ts if current_ts else True):
            ev = pending_notifications.pop(0)
            logging.info(f"{ev['timestamp']} -> Sending {ev['status']} notif with ID {ev['id']}")
            if ev['status'] == 'win':
                self.budget_engage -= ev['br_price']
                self.budget_depense += ev['br_price']
            else:
                self.budget_engage -= ev['br_price']

In [4]:
data = pd.read_csv('08-07-2020_06-08-2020.csv', index_col="timestamp_string", parse_dates=True)
data.index.names = ['Date']
data.head()

,identifiant,timestamp,nombre_impressions,prix_impression,prix_total,win,secondes_notif
Date,,,,,,,
2020-07-08 06:00:00,1,1.594181e+09,10,1,10,True,365
2020-07-08 06:00:00,2,1.594181e+09,1,1,1,True,361
2020-07-08 06:00:01,3,1.594181e+09,5,1,5,True,555
2020-07-08 06:00:01,4,1.594181e+09,1,1,1,True,645
2020-07-08 06:00:03,5,1.594181e+09,1,1,1,True,310


In [5]:
pacing = Algo(budget_quotidien=3000, nb_heures_jour=14)

In [6]:
achats = list()
remaining = list()
bt = list()
cible = list()
vt = list()
spent = list()
engaged = list()
pending_notifications = list()
day = 8
for current_ts, row in data.iterrows():
    # Send current notifications
    pacing.send_pending_notifications(current_ts)
    if current_ts.day != day:
        day = current_ts.day
        engaged[-1] = pacing.budget_engage
        pacing.budget_restant = pacing.budget_quotidien - (pacing.budget_engage + pacing.budget_depense)
        remaining[-1] = pacing.budget_restant
        spent[-1] = pacing.budget_depense
    
    # Receive BR
    decision = pacing.decision_achat_avec_vitesse(row['timestamp'], row['prix_total'])
    logging.info(f"{current_ts} -> sending BR")
    # Making a decision
    if decision[0]:
        #Buying
        logging.info(f"{current_ts} <- receiving buy")
        next_notif_ts = current_ts + timedelta(seconds=row['secondes_notif'])
        status = "win" if row['win'] else "lose"
        notif_id = uuid.uuid4()
        logging.info(f" | {current_ts} decided {status}, will be notified @ {next_notif_ts} with ID {notif_id}")
        pending_notifications.append({"timestamp": next_notif_ts, "status": status, 'br_price': row['prix_total'], 'id': notif_id})
        pending_notifications.sort(key=lambda x: x['timestamp'])
    cible.append(pacing.cible)
    bt.append(decision[1])
    vt.append(decision[2])
    achats.append(decision[0])
    remaining.append(pacing.budget_restant)
    spent.append(pacing.budget_depense)
    engaged.append(pacing.budget_engage)
# Send remaining notifications
pacing.send_pending_notifications()
# Update last row after sending last notifications
engaged[-1] = pacing.budget_engage
pacing.budget_restant = pacing.budget_quotidien - (pacing.budget_engage + pacing.budget_depense)
remaining[-1] = pacing.budget_restant
spent[-1] = pacing.budget_depense
data['cible'] = cible
data['bt'] = bt
data['vt'] = vt
data['achat'] = achats
data['remaining_budget'] = remaining
data['engaged_budget'] = engaged
data['spent_budget'] = spent

In [7]:
data.to_csv('pacing_08-07-2020_06-08-2020.csv', index=True)